$$
\begin{align*}
x(t) &= [\sqrt{D^2 - z^2}]\cos(t) + x_1\\
y(t) &= [\sqrt{D^2 - z^2}]\sin(t) + y_1\\

r(t) &= \sqrt{[D^2 - z^2]\cos^2(t) + x_1^2 + x_1[\sqrt{D^2 - z^2}]\cos(t) + [D^2 - z^2]\sin^2(t) + y_1^2 + y_1[\sqrt{D^2 - z^2}]\sin(t) + z^2}\\
&= \sqrt{[D^2 - z^2] + x_1^2 + y_1^2 + z^2 + [\sqrt{D^2 - z^2}](x_1\cos(t)+y_1\sin(t))}\\
&= \sqrt{D^2 + x_1^2 + y_1^2 + [\sqrt{D^2 - z^2}](x_1\cos(t)+y_1\sin(t))}\\

\frac{dx}{dt} &= -[\sqrt{D^2 - z^2}]\sin(t)\\
\frac{dy}{dt} &= [\sqrt{D^2 - z^2}]\cos(t)\\

{\begin{bmatrix} \frac{dx}{dt} \end{bmatrix}}^2 &= (D^2 - z^2)\sin^2(t)\\
{\begin{bmatrix} \frac{dy}{dt} \end{bmatrix}}^2 &= (D^2 - z^2)\cos^2(t)\\

{\begin{bmatrix} \frac{dx}{dt} \end{bmatrix}}^2 + {\begin{bmatrix} \frac{dy}{dt} \end{bmatrix}}^2 &= (D^2 - z^2)\sin^2(t) + (D^2 - z^2)\cos^2(t)\\
&= (D^2 - z^2)\\

\text{We have a density $\rho$, such that } \rho &= \rho(r),\\
\text{The mass around the circle } &= \int_0^{2\pi} \rho(r(t))\sqrt{{\begin{bmatrix} \frac{dx}{dt} \end{bmatrix}}^2 + {\begin{bmatrix} \frac{dy}{dt} \end{bmatrix}}^2} \; dt\\
\text{The mass around a sphere of radius D, } M(D) &= \int_{-D}^{D} \frac{D}{\sqrt{D^2 - z^2}} \int_0^{2\pi} \rho(r(t))\sqrt{{\begin{bmatrix} \frac{dx}{dt} \end{bmatrix}}^2 + {\begin{bmatrix} \frac{dy}{dt} \end{bmatrix}}^2} \; dt \: dz \\

&= \int_{-D}^{D} \frac{D}{\sqrt{D^2 - z^2}} \int_0^{2\pi} \rho(r(t))\sqrt{D^2 - z^2} \; dt \: dz \\
&= \int_{-D}^{D} \frac{D\sqrt{D^2 - z^2}}{\sqrt{D^2 - z^2}} \int_0^{2\pi} \rho(r(t)) \; dt \: dz \\
&= \int_{-D}^{D} D \int_0^{2\pi} \rho(r(t)) \; dt \: dz \\
\end{align*}
$$

$$
\begin{align*}
\text{For } \rho(r) &= c\text{, }\\

M(D) &= \int_{-D}^{D} D \int_0^{2\pi} c \; dt \: dz \\
&= \int_{-D}^{D} 2 \pi c D \: dz \\
&= 2 \pi c D^2 + 2 \pi c D^2\\
&= 4 \pi c D^2\\

\phi(D) &= \frac{G * 4 \pi c D^2}{D}\\
&= 4 \pi c D G\\
\end{align*}
$$

In [ ]:
import sim
import numpy as np
import matplotlib.pyplot as plt